In [1]:
import os
os.environ["PUBLIC_BI_BENCHMARK"] = "D:\\semantic_data_lake\\semantic_data_lake\\data\\benchmark\\"
import sys
sys.path.insert(0, "D:\\semantic_data_lake\\semantic_data_lake")
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, StructType, StructField
from pyspark.sql.functions import udf, col, pandas_udf, PandasUDFType, collect_list, count, avg, lit, mean, stddev, monotonically_increasing_id, row_number
from scipy.stats import wasserstein_distance
from numpy import asarray
from pyspark.sql.window import Window
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random

from tqdm import tqdm

from itertools import permutations, combinations, combinations_with_replacement
from sklearn.metrics import classification_report

from helper_functions import print_df_to_html, translate_header_file_to_list, variations, pair_permutations_ordered, translate_datatype_file_to_list, cast_datatypes, check_attribute_completeness, compare_schemas
from labeling_functions.similarity_calculations import appr3_calc_similarities, appr4_calc_similarities
from data_loader.adding_noise import add_additive_gaussian_noise
from data_loader.utils import load_public_bi_table
from data_loader.utils_for_pyspark import load_public_bi_tables_in_spark_temp_view
import pandas as pd
pd.options.display.max_columns = None
import json

C:\ProgramData\Anaconda3\envs\semantic_data_lake\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\semantic_data_lake\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\semantic_data_lake\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\semantic_data_lake\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Helper Functions

In [2]:
# classification report for labeling performance
def classification_report_labeling(df,cols_to_label, max_group_count:int):
    #col_to_label = "H"
    #max_group_count = 1
    true_labels = []
    pred_labels = []
    df = df.withColumn("EMD",df["EMD"].cast(DoubleType()))
    for column in cols_to_label:
        if df.select("*").where(col("INNER_ATTR") == column).count() == 0:
            continue
        pred_label = df.select("*").where(col("INNER_ATTR") == column).where(col("count") <= max_group_count).groupBy("OUTER","OUTER_ATTR","INNER","INNER_ATTR").avg("EMD").alias("avg(EMD)").sort("avg(EMD)").collect()[0]["OUTER_ATTR"]
        #print(pred_label)
        true_labels.append(column)
        pred_labels.append(pred_label)
    class_report_dic = classification_report(true_labels, pred_labels, output_dict=True)
    return class_report_dic


# Create Sprak Session

In [3]:
#spark.sparkContext.stop()
# create Spark Config
conf = SparkConf()
#conf.set("spark.executor.instances","2")
#conf.set("spark.executor.cores","2")
conf.set("spark.executor.memory", "8g")
conf.set("spark.driver.memory", "15g")
conf.set("spark.memory.offHeap.enabled", "true")
conf.set("spark.memory.offHeap.size", "16g")
#conf.set("spark.sql.execution.arrow.enabled", "true")
conf.setMaster("local[*]")
conf.setAppName("MLB-similarity-calc")
# create a SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
# create and register UDF-Function to calc EMD-Distance
@udf(returnType=FloatType())
def emd_UDF(col1, col2) -> FloatType:
    return float(wasserstein_distance(col1, col2))

spark.udf.register("emd_UDF", emd_UDF)

<function __main__.emd_UDF(col1, col2) -> pyspark.sql.types.FloatType>

# Read the tables into spark 

In [5]:
string_attributes, numeric_attributes = load_public_bi_tables_in_spark_temp_view("MLB")

Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458117263387 is not numeric or string
Calculation_40532458113208334 is not numeric or string
Calculation_40532458117263387 is not numeric or string
Calculation_40532458113208334 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_496521908233621517 is not numeric or string
Calculation_40532458112270348 is not numeric or string
Calculation_40532458116673561 is not numeric or string
Calculation_496521908233621517 is not numeric or string
Calculation_40532458112270348 is not numeric or string
Calculation_40532458116673561 is not numeric or string
Calculat

In [43]:
spark.sql("SELECT batter_name, parentteam, hr, x1b, x2b, x3b FROM MLB_1").show(50)

+------------------+----------+---+---+---+---+
|       batter_name|parentteam| hr|x1b|x2b|x3b|
+------------------+----------+---+---+---+---+
|     Chase Larsson|       ATL|  1|  7|  6|  0|
|     Kenny Fleming|       ATL|  0|  7|  0|  0|
|      Kurt Fleming|       ATL|  0| 13|  2|  1|
|      Kyle Kubitza|       ATL|  0| 11|  3|  3|
|   Nick DeSantiago|       ATL|  0|  5|  2|  0|
|      Adam Heisler|       CWS|  0|  2|  0|  0|
|      Brent Tanner|       CWS|  0|  3|  0|  0|
|    Carlos Sanchez|       CWS|  0|  1|  1|  0|
|       Cory Farris|       CWS|  1|  7|  1|  1|
|     Drew Thompson|       CWS|  0|  0|  0|  0|
|     Dusty Harvard|       CWS|  0|  5|  2|  1|
|     Jordan Keegan|       CWS|  0|  9|  0|  0|
|   Michael Johnson|       CWS|  0| 13|  2|  1|
|  Michael Schwartz|       CWS|  0|  3|  2|  0|
|    Sean O'Connell|       CWS|  0|  3|  1|  0|
|      Ariel Ovando|       HOU|  0|  8|  3|  2|
|         Chan Moon|       HOU|  0|  7|  1|  1|
|    Chase Davidson|       HOU|  4|  9| 

# Noise with Approach 3

In [ ]:
## APPROACH 3
# duration of about 7 hours
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.1
max_group_count = 3

df_noise = add_additive_gaussian_noise(
    table_name, data_perc_with_noise, random_seed, numeric_attributes)

numeric_attributes[table_name+"_noise"] = numeric_attributes[table_name]
string_attributes[table_name+"_noise"] = string_attributes[table_name]

df_noise.createOrReplaceTempView(table_name+"_noise")

#cols_included = ["H","X1B","X2B"]
# numeric cols not having just zeros as values
cols_included = ["AB", "AVG", "BABIP", "BIP", "FB", "GB", "GIDP", "HBP", "HR", "H", "ISO", "LD", "OBP",
                 "PA", "PU", "SF", "SH", "SLG", "SOL", "SOS", "SO", "TB", "X1B", "wOBA", "wRAA", "wRC", "year"]

for col_to_label in cols_included:
    true_labels = []
    predicted_labels = []
    df = appr3_calc_similarities(table_name+"_noise", table_name, string_attributes, numeric_attributes, max_group_count, [col_to_label])
    df_sorted = df.select("*").sort("avg(EMD)").collect()
    true_labels.append(df_sorted[0]["INNER_ATTR"])
    predicted_labels.append(df_sorted[0]["OUTER_ATTR"])


    labeling_results={
        "labeled_table":table_name+"_noise",
        "true_labels": true_labels,
        "predicted_labels":predicted_labels
    }
    with open(f"results_emd/noise_compare/appro3/appro3_noise{data_perc_with_noise}_mgrc{max_group_count}_col_{col_to_label}.json", "w") as outfile:
        json.dump(labeling_results, outfile)

In [ ]:
with open(f"results_emd/noise_compare/appro3/appro3_noise{data_perc_with_noise}_mgrc{max_group_count}_col_{col_to_label}.json", "w") as outfile:
        json.dump(labeling_results, outfile)

In [ ]:
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.1
max_group_count = 3

df_noise = add_additive_gaussian_noise(
    table_name, data_perc_with_noise, random_seed, numeric_attributes)

numeric_attributes[table_name+"_noise"] = numeric_attributes[table_name]
string_attributes[table_name+"_noise"] = string_attributes[table_name]

df_noise.createOrReplaceTempView(table_name+"_noise")

In [ ]:
df = appr3_calc_similarities(table_name+"_noise", table_name, string_attributes, numeric_attributes, 3, ["H"])
df.select("*").sort("EMD")

In [ ]:
df.show(50)

# Doing Matching and Labeling of cols based on similarity metrics

In [ ]:
sqlOuter = spark.sql("SELECT * FROM MLB_1")
wasserstein_distance(sqlOuter.select(collect_list("H")).collect()[0][0], sqlOuter.select(collect_list("X1B")).collect()[0][0])

In [ ]:
# funtion to calc labeling performance with approach 1
def appr1_calc_similarities(outer, inner, sel_attr):
    join_attribute = list(set(string_attributes[inner]))
    intersecting_attr = list(set(numeric_attributes[inner]))
    sqlOuter = spark.sql("SELECT * FROM " + outer)
    # filter out null tupels with null values
    sqlOuter = sqlOuter.dropna(subset=list(
        map(lambda cur_col: "`{cur_col}`".format(cur_col=cur_col),
            sqlOuter.columns)))
    sqlInner = spark.sql("SELECT * FROM "+inner)
    # filter out null tupels with null values
    sqlInner = sqlInner.dropna(subset=list(
        map(lambda cur_col: "`{cur_col}`".format(cur_col=cur_col),
            sqlInner.columns)))
    attr_variations = pair_permutations_ordered(intersecting_attr)
    # selsect specific attr_variation with a specific attribute included
    #sel_attr = ['H','BB','X1B','X2B']
    sel_attr = sel_attr
    sel_attr_variations = list(
         filter(lambda x: x[1] in sel_attr, attr_variations))

    result = []         
    for index_attr, curr_item in enumerate(sel_attr_variations):
        print(str(index_attr) + "/" + str(len(sel_attr_variations)))
        first_attr = curr_item[0]
        second_attr = curr_item[1]
        emd = wasserstein_distance(sqlOuter.select(collect_list(f"`{first_attr}`")).collect()[0][0],sqlInner.select(collect_list(f"`{second_attr}`")).collect()[0][0])
        result.append([outer, first_attr, inner, second_attr, float(emd),0])
    resultDF = spark.createDataFrame(result).toDF("OUTER","OUTER_ATTR","INNER","INNER_ATTR","EMD","count")
    return resultDF

In [ ]:
# function to calc labeling performance with approach 3
def appr3_calc_similarities(outer, inner, sel_attr):
    outer = outer
    inner = inner
    join_attributes = list(set(string_attributes[inner]))
    join_condition = "ON (" + " AND ".join(map(lambda join_att : "o.`{join_att}` = i.`{join_att}`".format(join_att=join_att) ,\
                                           join_attributes))
    intersecting_attr = list(set(numeric_attributes[inner]))
    #print(intersecting_attr)
    #create projection list
    projection_list = " , ".join(
        map(lambda attr: "o.`{attr}` as `{attr}`".format(attr=attr),
            join_attributes)
    ) + " , " + " , ".join(
        map(
            lambda attr: "o.`{attr}` as `o.{attr}` , i.`{attr}` as `i.{attr}`".
            format(attr=attr), intersecting_attr))
    sqlDF = spark.sql("SELECT "+projection_list+" FROM " +outer +" o JOIN "+ \
                            inner+ " i " + join_condition+")")
    # filter out null tupels with null values
    sqlDF = sqlDF.dropna(subset=list(
        map(lambda cur_col: "`{cur_col}`".format(cur_col=cur_col),
            sqlDF.columns)))
    # calculates null values in the table
    #sqlDF.select([count(when(isnan(f"`{c}`") | col(f"`{c}`").isNull(), c)).alias(f"`{c}`") for c in sqlDF.columns]).show()

    # calcultes basic statisitc for the attributes
    #print_df_to_html(sqlDF.describe())

    attr_variations = pair_permutations_ordered(intersecting_attr)
    #print(attr_variations)

    # selsect specific attr_variation with a specific attribute included
    #sel_attr = ['H','BB','X1B','X2B']
    sel_attr = sel_attr
    sel_attr_variations = list(
         filter(lambda x: x[1] in sel_attr, attr_variations))
    #print(sel_attr_variations)
    
    #sel_attr_variations = [['H', "H"]]

    for index_attr, curr_item in enumerate(sel_attr_variations):
        print(str(index_attr) + "/" + str(len(sel_attr_variations)))
        first_attr = curr_item[0]
        second_attr = curr_item[1]
        #print(first_attr)
        #print(second_attr)
        if index_attr == 0:
            curDF = sqlDF.groupby(join_attributes).agg(
                emd_UDF(collect_list("`o.{first_attr}`".format(first_attr=first_attr)),
                        collect_list("`i.{second_attr}`".format(second_attr=second_attr))).alias("EMD"),
                count("`i.{second_attr}`".format(second_attr=second_attr)).alias("count")).select(col("EMD"), col("count"))
            curDF = curDF.withColumn("OUTER", lit(outer)).withColumn(
                "OUTER_ATTR",
                lit(first_attr)).withColumn("INNER", lit(inner)).withColumn(
                    "INNER_ATTR", lit(second_attr))
        else:
            newDF = sqlDF.groupby(join_attributes).agg(
                emd_UDF(collect_list("`o.{first_attr}`".format(first_attr=first_attr)),
                        collect_list("`i.{second_attr}`".format(second_attr=second_attr))).alias("EMD"),
                count("`i.{second_attr}`".format(second_attr=second_attr)).alias("count")).select(col("EMD"), col("count"))
            newDF = newDF.withColumn("OUTER", lit(outer)).withColumn(
                "OUTER_ATTR",
                lit(first_attr)).withColumn("INNER", lit(inner)).withColumn(
                    "INNER_ATTR", lit(second_attr))
            curDF = curDF.union(newDF)
    return curDF
    # curDF.write.format("csv").mode("overwrite").option("header", "true").save(
    #     "/HdiNotebooks/semantic_data_lake/results/emd_results_dist_sep_inst_{outer}_{inner}".format(
    #         outer=outer, inner=inner))

In [ ]:
# function to calc labeling performance with approach 4
def appr4_calc_similarities(outer, inner, sel_attr, max_group_counts):
    outer = outer
    inner = inner
    # find matching attributes to compare
    join_attributes = list(set(string_attributes[inner]))
    join_condition = "ON (" + " AND ".join(map(lambda join_att : "o.`{join_att}` = i.`{join_att}`".format(join_att=join_att) ,\
                                           join_attributes))
    intersecting_attr = list(set(numeric_attributes[inner]))
    #print(intersecting_attr)
    #create projection list
    projection_list = " , ".join(
        map(lambda attr: "o.`{attr}` as `{attr}`".format(attr=attr),
            join_attributes)
    ) + " , " + " , ".join(
        map(
            lambda attr: "o.`{attr}` as `o.{attr}` , i.`{attr}` as `i.{attr}`".
            format(attr=attr), intersecting_attr))
    sqlDF = spark.sql("SELECT "+projection_list+" FROM " +outer +" o JOIN "+ \
                            inner+ " i " + join_condition+")")
    # filter out null tupels with null values
    sqlDF = sqlDF.dropna(subset=list(
        map(lambda cur_col: "`{cur_col}`".format(cur_col=cur_col),
            sqlDF.columns)))

    result_list=[]
    #for max_group_count in list(range(1,9,2))+list(range(10,150,20)):
    for max_group_count in max_group_counts:
        print(max_group_count)
        
        # groupby string attribute and filter out instances which only consider specific times
        sqlDF_instances_to_consider = sqlDF.groupby(join_attributes).agg(count(join_attributes[0]).alias("count")).where(col("count") <= max_group_count)
        resSqlDF = sqlDF.join(sqlDF_instances_to_consider, on=join_attributes, how='inner')

        attr_variations = pair_permutations_ordered(intersecting_attr)
        #print(attr_variations)

        # selsect specific attr_variation with a specific attribute included
        sel_attr = sel_attr
        sel_attr_variations = list(
            filter(lambda x: x[1] in sel_attr, attr_variations))
        #print(sel_attr_variations)
        #print(sel_attr_variations)
        #print(len(sel_attr_variations))

        #sel_attr_variations = [['H', "H"]]
        
        for index_attr, curr_item in enumerate(sel_attr_variations):
            print(str(index_attr)+"/"+str(len(sel_attr_variations)))
            first_attr = curr_item[0]
            second_attr = curr_item[1]
            # print(first_attr)
            # print(second_attr)
            emd = resSqlDF.select(emd_UDF(collect_list("`o.{first_attr}`".format(first_attr=first_attr)),collect_list("`i.{second_attr}`".format(second_attr=second_attr))).alias("EMD")).collect()[0]["EMD"]
            #print([outer,first_attr, inner, second_attr, max_group_count, float(emd)])
            result_list.append([outer,first_attr, inner, second_attr, max_group_count, float(emd)])
        resultDF = spark.createDataFrame(result_list).toDF("OUTER","OUTER_ATTR","INNER","INNER_ATTR","COUNT","EMD")
        #resultDF.write.format("csv").mode("overwrite").option("header", "true").save("/HdiNotebooks/semantic_data_lake/results/emd_results_sep_instances_appr4/{outer}_{inner}/emd_results_dist_sep_inst_appr4_maxgr{max_group_count}_{first_attr}_{second_attr}".format(outer=outer, inner=inner, max_group_count=max_group_count, first_attr=first_attr, second_attr=second_attr))
    return resultDF


In [ ]:
curDF = appr4_calc_similarities()
#classification_report_labeling(curDF, ["H"], 1)

In [ ]:
df = curDF.select("*").where(col("INNER_ATTR") == "H").where(col("count") <= 1).groupBy("OUTER","OUTER_ATTR","INNER","INNER_ATTR").avg("EMD").alias("avg(EMD)").sort("avg(EMD)")
print(df.collect()[0]["OUTER_ATTR"])
print(df.collect()[0]["avg(EMD)"])

In [ ]:
curDF.select("*").where(col("INNER_ATTR") == "H").where(col("count") <= 1).groupBy("OUTER","OUTER_ATTR","INNER","INNER_ATTR").avg("EMD").alias("avg(EMD)").sort("avg(EMD)").show(50)

# Adding Noise: exchange colum values

In [ ]:
ac_table = "MLB_1"
# first try to add noise to one column by selecting values from other column and replace the values
df = spark.sql("SELECT * from {ac_table}".format(ac_table=ac_table))
df_noise = df

# want bring 10% noise in it
noise_frac = 0.15
# calc how many values are needed to bring x% noise in one column
#number_of_noisevalues = int(df.count()*noise_frac)


# changing values for H for test
dic_noise = {}
# adding the string attributes initialy
for string_attribute in string_attributes["MLB_1"]:
    dic_noise[string_attribute] = []
for index,numeric_attribute_1 in enumerate(numeric_attributes["MLB_1"]):
    print(numeric_attribute_1)
    dic_noise[numeric_attribute_1] = []
    if index > 0:
        for string_attribute in string_attributes["MLB_1"]:
            dic_noise[string_attribute].extend(df.select(col("`"+string_attribute+"`")).sample(noise_frac/(len(numeric_attributes["MLB_1"])-1),0).rdd.flatMap(list).collect())
    for numeric_attribute_2 in numeric_attributes["MLB_1"]:
        #print(numeric_attribute_1)
        if numeric_attribute_2 == numeric_attribute_1:
            continue
        dic_noise[numeric_attribute_1].extend(df.select(col("`"+numeric_attribute_2+"`")).sample(noise_frac/(len(numeric_attributes["MLB_1"])-1),0).rdd.flatMap(list).collect())


pd.DataFrame(dic_noise).to_csv(file_path+"MLB_1_noise"+str(noise_frac*100)+".csv",index=False)

In [ ]:
len(dic_noise["batter_name"])

In [ ]:
print(df.count())

split = df.randomSplit([0.8,0.2],0)
print(split[0].count()/df.count())
print(split[1].count()/df.count())

# Adding noise similiar to Valentine

In [ ]:
table_name = "MLB_1"
random_plus_minus_range = 10

df = spark.sql("SELECT * from MLB_1")
# adding noise
for numeric_attribute in numeric_attributes[table_name]:
    #print(numeric_attribute)
    #if numeric_attribute != "AB":
    #    break
    
    df_stats = spark.sql("SELECT mean(`{numeric_attribute}`) as mean, stddev(`{numeric_attribute}`) as stddev from {table_name}".format(
        numeric_attribute=numeric_attribute, table_name=table_name)).collect()[0]
    df_col_mean = df_stats["mean"]
    df_col_stddev = df_stats["stddev"]
    #print(df_col_mean, df_col_stddev)
    df_col_mean_new = float(df_col_mean) + float(df_col_mean) * random.uniform(-random_plus_minus_range, random_plus_minus_range)/100
    df_col_stddev_new = float(df_col_stddev) + float(df_col_stddev) * random.uniform(-random_plus_minus_range, random_plus_minus_range)/100
    #print(df_col_mean_new, df_col_stddev_new)
    new_col = np.random.normal(df_col_mean_new, df_col_stddev_new, df.count())
    #print(df.count())
    #print(len(new_col))
    #print(new_col.mean())
    df_col_new = spark.createDataFrame([(float(l),) for l in new_col], schema=StructType([
                                       StructField("new_col", FloatType())]))
    df_col_new = df_col_new.withColumn('row_index', row_number().over(
        Window.orderBy(monotonically_increasing_id())))
    df = df.withColumn('row_index', row_number().over(
        Window.orderBy(monotonically_increasing_id())))

    df = df.join(df_col_new, on=["row_index"]).drop("row_index")
    df = df.withColumn("{numeric_attribute}".format(numeric_attribute=numeric_attribute), df.new_col).drop("new_col")
df.createOrReplaceTempView(table_name+"_noise")


In [ ]:
spark.sql("SELECT mean(H) from MLB_1").show()

In [ ]:
spark.sql("SELECT mean(H) from MLB_1_noise").show()

# Additive Gaussian Noise with N(0,stdd)
- stdd => standard deviation of the specific column, where noise should be added

In [ ]:
# random_seed = 0
# np.random.seed(random_seed)
# table_name = "MLB_1"
# data_perc_with_noise = 0.1

# df_original = spark.sql(f"SELECT * from {table_name}")

def add_additive_gaussian_noise(table_name, data_perc_with_noise, random_seed):
  df_original = spark.sql(f"SELECT * from {table_name}")
  df_without_noise, df_to_add_noise = df_original.randomSplit([1-data_perc_with_noise, data_perc_with_noise],random_seed)
  #print(df.count(),df_without_noise.count(), df_to_add_noise.count())
  df_to_add_noise_pd = df_to_add_noise.toPandas()

  # adding noise to every numeric column
  for numeric_attribute in numeric_attributes[table_name]:
      #print(numeric_attribute)
      #if numeric_attribute != "AB":
        #  break
      df_stats = df_original.select(mean(f"`{numeric_attribute}`").alias("mean"), stddev(f"`{numeric_attribute}`").alias("stddev")).collect()[0]
      df_col_mean = df_stats["mean"]
      df_col_stddev = df_stats["stddev"]

      for index, row in df_to_add_noise_pd.iterrows():
          if row[f"{numeric_attribute}"] != None: # filter None values
            df_to_add_noise_pd.at[index, f"{numeric_attribute}"] = float(row[f"{numeric_attribute}"]) + float(np.random.normal(0,df_col_stddev,1)[0])

  df_to_add = spark.createDataFrame(df_to_add_noise_pd)
  df_noise_full = df_without_noise.union(spark.createDataFrame(df_to_add_noise_pd))
  return df_noise_full

# Doing Matching and Labeling with noisy data (additive noise with N(0,stdd))

In [ ]:
## APPROACH 1
# duration of about 7 hours
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.5
max_group_count = 0


for data_perc_with_noise in [0.01]:
    df_noise = add_additive_gaussian_noise(table_name, data_perc_with_noise, random_seed)

    df_noise.createOrReplaceTempView(table_name+"_noise")

    #cols_included = ["H","X1B","X2B"]
    # numeric cols not having just zeros as values
    cols_included = ["AB", "AVG", "BABIP", "BIP", "FB", "GB", "GIDP", "HBP", "HR", "H", "ISO", "LD", "OBP",
            "PA", "PU", "SF", "SH", "SLG", "SOL", "SOS", "SO", "TB", "X1B", "wOBA", "wRAA", "wRC", "year"]

    #cols_included = ["FB", "GB", "HR", "H", "LD", "X1B","X2B", "X3B","wRC", "year"]
    #cols_included = ["H","X1B","X2B","X3B","wRC"]

    df = appr1_calc_similarities(table_name+"_noise", table_name, cols_included)

    class_report = classification_report_labeling(df, cols_included, max_group_count)

    pickle.dump(class_report, open(f"results_emd/noise_compare/appro1_noise{data_perc_with_noise}_mgrc{max_group_count}_all_columns.p", "wb"))
    #pickle.dump(class_report, open(f"semantic_data_lake/semantic_data_lake/results/noise_compare/appro3_noise{data_perc_with_noise}_mgrc{max_group_count}_all_columns.p", "wb"))

In [ ]:
## APPROACH 3 
# duration of about 7 hours
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.5
max_group_count = 3

df_noise = add_additive_gaussian_noise(table_name, data_perc_with_noise, random_seed)

df_noise.createOrReplaceTempView(table_name+"_noise")

#cols_included = ["H","X1B","X2B"]
# numeric cols not having just zeros as values
cols_included = ["AB", "AVG", "BABIP", "BIP", "FB", "GB", "GIDP", "HBP", "HR", "H", "ISO", "LD", "OBP",
        "PA", "PU", "SF", "SH", "SLG", "SOL", "SOS", "SO", "TB", "X1B", "wOBA", "wRAA", "wRC", "year"]

#cols_included = ["FB", "GB", "HR", "H", "LD", "X1B","X2B", "X3B","wRC", "year"]
#cols_included = ["H","X1B","X2B","X3B","wRC"]

df = appr3_calc_similarities(table_name+"_noise", table_name, cols_included)

class_report = classification_report_labeling(df, cols_included, max_group_count)

pickle.dump(class_report, open(f"results_emd/noise_compare/appro3_noise{data_perc_with_noise}_mgrc{max_group_count}_all_columns.p", "wb"))

In [ ]:
## Approach 4
# duration of about 
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.2
max_group_count = [3]

for data_perc_with_noise in [0.62,0.65,0.67,0.75,0.95]:
    
    df_noise = add_additive_gaussian_noise(table_name, data_perc_with_noise, random_seed)

    df_noise.createOrReplaceTempView(table_name+"_noise")

    #cols_included = ["H","X1B","X2B"]
    # numeric cols not having just zeros as values
    #cols_included = ["AB", "AVG", "BABIP", "BIP", "FB", "GB", "GIDP", "HBP", "HR", "H", "ISO", "LD", "OBP",
    #        "PA", "PU", "SF", "SH", "SLG", "SOL", "SOS", "SO", "TB", "X1B", "wOBA", "wRAA", "wRC", "year"]

    #cols_included = ["FB", "GB", "HR", "H", "LD", "X1B","X2B", "X3B","wRC", "year"]
    cols_included = ["H","X1B","X2B","X3B","wRC"]

    df = appr4_calc_similarities(table_name+"_noise", table_name, cols_included, max_group_count)

    class_report = classification_report_labeling(df, cols_included, max_group_count[0])

    pickle.dump(class_report, open(f"results_emd/noise_compare/appro4_noise{data_perc_with_noise}_mgrc{max_group_count[0]}.p", "wb"))


In [ ]:
## Approach 4
# duration of about 
random_seed = 0
np.random.seed(random_seed)
table_name = "MLB_1"
data_perc_with_noise = 0.2
max_group_count = [3]

for data_perc_with_noise in [0.01,0.1,0.2,0.3,0.4,0.6,0.7,0.8,0.9,0.99]:
#for data_perc_with_noise in [0.1]:
    df_noise = add_additive_gaussian_noise(table_name, data_perc_with_noise, random_seed)

    df_noise.createOrReplaceTempView(table_name+"_noise")

    #cols_included = ["H","X1B","X2B"]
    # numeric cols not having just zeros as values
    cols_included = ["AB", "AVG", "BABIP", "BIP", "FB", "GB", "GIDP", "HBP", "HR", "H", "ISO", "LD", "OBP",
            "PA", "PU", "SF", "SH", "SLG", "SOL", "SOS", "SO", "TB", "X1B", "wOBA", "wRAA", "wRC", "year"]

    #cols_included = ["FB", "GB", "HR", "H", "LD", "X1B","X2B", "X3B","wRC", "year"]
    #cols_included = ["H","X1B","X2B","X3B","wRC"]

    df = appr4_calc_similarities(table_name+"_noise", table_name, cols_included, max_group_count)

    class_report = classification_report_labeling(df, cols_included, max_group_count[0])

    pickle.dump(class_report, open(f"results_emd/noise_compare/appro4_noise{data_perc_with_noise}_mgrc{max_group_count[0]}_all_columns.p", "wb"))


# Doing Matching and Labeling with noisy Data

In [ ]:
random_seed = 0
noise_perc = 0.01 # 0.01 => 1% noise adding
table_name = "MLB_1"
# Build the datafram with noise
# first cut the noise_percentage from the original data
projection_list = " , ".join(
        map(lambda attr: "`{attr}`".format(attr=attr),
            numeric_attributes[table_name])
    )
df_noise_comp = spark.sql("SELECT "+projection_list +" FROM {table}".format(table=table)).randomSplit([(1-noise_perc),noise_perc],random_seed)[0]
#df_noise = df_noise.select(numeric_attributes[table_name])
# read the random selected noise
data_file = file_path + table_name + "_noise"+str(noise_perc*100) + ".csv"

header_file = file_path + "samples/" + table_name + ".header.csv"
datatype_file = file_path + "samples/" + table_name + ".datatypes.csv"
# create a DataFrame using an ifered Schema
df_noise_to_add = spark.read.option("header", "true") \
.option("inferSchema", "true") \
.option("delimiter", ",") \
.csv(data_file).toDF(*numeric_attributes[table_name])

print(df_noise.count())
print(df_noise_to_add.count())
df_noise = df_noise.union(df_noise_to_add)
print(df_noise.count())

df_noise.createOrReplaceTempView(table_name+"_noise")

outer = table_name
inner = table_name+"_noise"



In [ ]:
print(spark.sql("SELECT * from MLB_1").count())
print(spark.sql("SELECT * from MLB_1_noise").count())